In [200]:
import matplotlib.pyplot as plt
import pandas as pd
import requests
import json
import datetime
from pytz import timezone

In [201]:
API_URL = "https://www.alphavantage.co/query"

In [202]:
with open('../api.txt','r') as file:
    apikey = file.read().replace('\n','')

Formatting current time

In [203]:
fmt = "%a %b %d %H:%M:%S %Z %Y"
now_time = datetime.datetime.now(timezone('US/Pacific'))
time = now_time.strftime(fmt)

### Following are the functions used

In [204]:
def increase_or_decrease(x):
    if  (x>0):
        return '+'
    else :
        return '-'

In [205]:
def get_stock_data(symbol):
    data_global_quote = {
        "function" : "GLOBAL_QUOTE",
        "symbol" : symbol,
        "apikey" : apikey
    }

    data_symbol_search = {
        "function" : "SYMBOL_SEARCH",
        "keywords" : symbol,
        "apikey" : apikey
    }
    response = requests.get(API_URL, params=data_global_quote)
    response_symbol_search = requests.get(API_URL, params=data_symbol_search)
    response_dict = response.json(),
    response_symbol_search_dict = response_symbol_search.json()
    
   
    if(response.status_code == 200 and response_symbol_search.status_code == 200):
        
        found, full_name = get_full_name(symbol,response_symbol_search_dict)
        if(found == False):
            stock_response = "Symbol Not Found"
            return stock_response
            
        #Symbol found
        stock_response = {
            "response_dict" : response_dict,
            "response_symbol_search_dict" : response_symbol_search_dict,
            "full_name" : full_name
        }
    else:
        stock_response = "Connectivity Error Encountered!"
    
    return stock_response

In [206]:
def get_full_name(symbol, response_symbol_search_dict):
    found = False
    full_name = ''
    for x in response_symbol_search_dict['bestMatches']:
        if(symbol == x['1. symbol'] and found != True):
            #print(x)
            found = True
            full_name = x['2. name']
            break
    if(found == False):
        print('Check value entered')
    
    return found,full_name

In [207]:
"""We received data in json format and converted it to a dictionary, 
    we can now parse it to get real time values """ 
def parse_stock_data(stock_response):
    #print("printing response dict"+str(stock_response['response_dict']))
    
    #print("stock price" + stock_response['response_dict']['Global Quote'][0])
    stock_price = stock_response['response_dict'][0]['Global Quote']['05. price']
    value_changes_sign = increase_or_decrease(float(stock_response['response_dict'][0]['Global Quote']['09. change']))
    value_changes = stock_response['response_dict'][0]['Global Quote']['09. change']
    percentage_change_sign = increase_or_decrease(float(stock_response['response_dict'][0]['Global Quote']['10. change percent'].split('%')[0]))
    percentage_change = stock_response['response_dict'][0]['Global Quote']['10. change percent']
    full_name_company = stock_response['full_name']
   
  
    data = {
        "stock_price" : stock_price,
        "value_changes" : value_changes,
        "value_changes_sign" : value_changes_sign,
        "percentage_change" : percentage_change,
        "percentage_change_sign" : percentage_change_sign,
        "full_name_company" : full_name_company
    }
    
    return data

In [211]:
def print_stock_data(data,symbol):
    print('\nOutput : ')
    print(time)
    print(data['full_name_company']+ ' ('+symbol+')')
    print(data['stock_price']+data['value_changes_sign']+data['value_changes']
         + data['percentage_change_sign']+data['percentage_change'])

### Ask user to input

In [213]:
while True:
    symbol = input('Enter A stock symbol or quit to exit: ').upper()
    if symbol == "QUIT":
        print("Thanks for using this calculator! See you soon!")
        break
    else:
        stock_response = get_stock_data(symbol)
        #print(stock_response)
        if(stock_response == "Symbol Not Found" or stock_response == "Connectivity Error Encountered!"):
            print(stock_response)
            continue
        data = parse_stock_data(stock_response)
        print_stock_data(data,symbol)
        print('\n')

Enter A stock symbol or quit to exit:  quit


Thanks for using this calculator! See you soon!
